#### Configurations

In [1]:
GITLAB_URL='https://gitlab.com'
GITLAB_ACCESS_TOKEN=''
FILTER_DATE_BEGIN='2020-05-01'
FILTER_DATE_END='2020-05-31'
FILTER_AUTHOR=None
FILTER_ONLY_MEMBER=True
FILTER_SEARCH='Bem'
DEBUG=True

### Fetch data from Gitlab

In [2]:
import pandas as pd
import gitlab
import modules.gitlabspent as gs
gl = gitlab.Gitlab(GITLAB_URL, private_token=GITLAB_ACCESS_TOKEN)

In [3]:
entries = gs.fetch_time_entries(gl, filter_by_author=FILTER_AUTHOR, filter_by_date_begin=FILTER_DATE_BEGIN, filter_by_date_end=FILTER_DATE_END, filter_by_project_membership=FILTER_ONLY_MEMBER, filter_by_search=FILTER_SEARCH, debug=DEBUG)
print('%s time entries fetched' %(len(entries)))

Getting projects. search=Bem only_member=True

Getting issues for project=Bem Promotions 
Getting issues for project=Bem MySQL 
Getting issues for project=Bem Frontend . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Getting issues for project=Bem Backend . 
Getting issues for project=Bem estar Saúde . 2 time entries fetched


In [4]:
df = pd.DataFrame()
for e in entries[0]:
  df = df.append(e, ignore_index=True)

## Raw spend table

In [5]:
df = df.sort_values(['date','note_author'])
df2 = df.astype({"duration": int, "issue_iid": int})
# df2['duration'] = df2['duration'].map(gs.format_duration)
df2

,date,duration,issue_iid,issue_title,note_author
11,2020-05-02,9600,46,Esqueci/lembrar senha e login,danismerim
13,2020-05-02,1500,33,Nota do profissional - padrão: 100%. A partir ...,danismerim
14,2020-05-09,600,22,"Cadastro de perfil profissional: incluir ""outr...",danismerim
12,2020-05-12,3600,46,Esqueci/lembrar senha e login,danismerim
10,2020-05-14,1500,59,Não encontra profissionais na barra de pesquisa,danismerim
17,2020-05-16,6000,13,Enviar e-mail para o participante ou fornecedo...,danismerim
5,2020-05-17,3600,77,"""Fale com bem"" não está enviando email para fa...",danismerim
9,2020-05-17,300,73,"Tirar opção sexo ""outro"" ao criar cadastro",danismerim
16,2020-05-17,7200,20,Validar email ao criar uma conta,danismerim
8,2020-05-18,1200,74,Não abriu agenda aos domingos,danismerim


# Time/cost by issue

In [38]:
COST_PER_HOUR=50

df3 = df2.groupby(['issue_iid','issue_title'])
df4 = df3['duration'].sum()
df5 = pd.DataFrame(df4)
df5 = df5.sort_values(['duration','issue_iid'], ascending=False)
df5['duration'] = df5['duration']
df5['cost'] = df5['duration'] * (COST_PER_HOUR/3600)

df6 = df5.copy()
df6['cost'] = df6['cost'].apply(lambda x: '%.2f' %x)
df6['duration'] = df6['duration'].apply(gs.format_duration)

print('Period: %s to %s' %(FILTER_DATE_BEGIN, FILTER_DATE_END))
print(df6)
print('')
print('Total cost:     %.2f' %df5['cost'].sum())
print('Total duration: %s' %gs.format_duration(df5['duration'].sum()))

Period: 2020-05-01 to 2020-05-31
                                                             duration    cost
issue_iid issue_title                                                        
46        Esqueci/lembrar senha e login                        3h 40m  183.33
20        Validar email ao criar uma conta                      2h 0m  100.00
13        Enviar e-mail para o participante ou fornecedor...   1h 40m   83.33
77        "Fale com bem" não está enviando email para fal...    1h 0m   50.00
59        Não encontra profissionais na barra de pesquisa      0h 25m   20.83
33        Nota do profissional - padrão: 100%. A partir d...   0h 25m   20.83
22        Cadastro de perfil profissional: incluir "outro...   0h 25m   20.83
75        ícones nos emails chegam quebrados                   0h 20m   16.67
74        Não abriu agenda aos domingos                        0h 20m   16.67
85        BUG - tem horário disponível, mas não agenda. N...   0h 15m   12.50
82        PWA - ícone errado   

# Total by user

In [37]:
df3 = df2.groupby(['note_author'])
df4 = df3['duration'].sum()
df5 = pd.DataFrame(df4)
df5 = df5.sort_values(['duration'], ascending=False)
df5['duration'] = df5['duration']
df5['cost'] = df5['duration'] * (COST_PER_HOUR/3600)
df5['cost'] = df5['cost'].apply(lambda x: '%.2f' %x)
df5['duration'] = df5['duration'].apply(gs.format_duration)
df5

,duration,cost
note_author,,
danismerim,11h 15m,1687.50
flaviostutz,0h 10m,25.00


In [40]:
COST_PER_HOUR=50

df3 = df2.groupby(['note_author','issue_iid','issue_title'])
df4 = df3['duration'].sum()
df5 = pd.DataFrame(df4)
df5 = df5.sort_values(['duration','issue_iid'], ascending=False)
df5['duration'] = df5['duration']
df5['cost'] = df5['duration'] * (COST_PER_HOUR/3600)

df5['cost'] = df5['cost'].apply(lambda x: '%.2f' %x)
df5['duration'] = df5['duration'].apply(gs.format_duration)

df5

duration  \
note_author issue_iid issue_title                                                   
danismerim  46        Esqueci/lembrar senha e login                        3h 40m   
            20        Validar email ao criar uma conta                      2h 0m   
            13        Enviar e-mail para o participante ou fornecedor...   1h 40m   
            77        "Fale com bem" não está enviando email para fal...    1h 0m   
            59        Não encontra profissionais na barra de pesquisa      0h 25m   
            33        Nota do profissional - padrão: 100%. A partir d...   0h 25m   
            22        Cadastro de perfil profissional: incluir "outro...   0h 25m   
            75        ícones nos emails chegam quebrados                   0h 20m   
            74        Não abriu agenda aos domingos                        0h 20m   
            85        BUG - tem horário disponível, mas não agenda. N...   0h 15m   
            82        PWA - ícone errado                                   0h 15m   
            86        Perfil profissional - meus dados - edição não s...   0h 10m   
            83        BUG tela branca depois de criar novo perfil          0h 10m   
flaviostutz 81        PWA - as vezes a tela fica branca                    0h 10m   
danismerim  76        adicionar informação do profissional no email         0h 5m   
            73        Tirar opção sexo "outro" ao criar cadastro            0h 5m   

                                                                            cost  
note_author issue_iid issue_title                                                 
danismerim  46        Esqueci/lembrar senha e login                       183.33  
            20        Validar email ao criar uma conta                    100.00  
            13        Enviar e-mail para o participante ou fornecedor...   83.33  
            77        "Fale com bem" não está enviando email para fal...   50.00  
            59        Não encontra profissionais na barra de pesquisa      20.83  
            33        Nota do profissional - padrão: 100%. A partir d...   20.83  
            22        Cadastro de perfil profissional: incluir "outro...   20.83  
            75        ícones nos emails chegam quebrados                   16.67  
            74        Não abriu agenda aos domingos                        16.67  
            85        BUG - tem horário disponível, mas não agenda. N...   12.50  
            82        PWA - ícone errado                                   12.50  
            86        Perfil profissional - meus dados - edição não s...    8.33  
            83        BUG tela branca depois de criar novo perfil           8.33  
flaviostutz 81        PWA - as vezes a tela fica branca                     8.33  
danismerim  76        adicionar informação do profissional no email         4.17  
            73        Tirar opção sexo "outro" ao criar cadastro            4.17